In [ ]:
from obspy import read, read_inventory, UTCDateTime
from obspy.geodetics import locations2degrees, gps2dist_azimuth
import matplotlib.pyplot as plt
from obspy.taup import TauPyModel
import numpy as np
from obspy.clients.fdsn import Client
from tqdm import tqdm
from obspy.signal.filter import envelope as env
from matplotlib import cm
from cmcrameri import cm as scm6
import matplotlib.patheffects as pe

In [ ]:
# Define event origin
ev_lat, ev_lon =  72.5, -27.3
ot = UTCDateTime(2023, 9, 16, 12, 35)

In [ ]:
# Use travel-time model(?)
model = TauPyModel(model="iasp91")

In [ ]:
# Read in raw waveforms and station metadata
st = read("../../../../../mdl/waveforms/*.mseed")
st.merge(fill_value=0)
inv = read_inventory("../../../../../mdl/stations/*")

In [ ]:
# Pre-process the waveforms
st2 = st.copy()
for n in tqdm(range(len(st2))):
    st2[n].trim(ot-3600, ot+(5*24*60*60))
    if st2[n].stats.sampling_rate == 250:
        st2[n].decimate(5)
        st2[n].decimate(5)
        st2[n].decimate(10)
        st2[n].decimate(10)
    elif st2[n].stats.sampling_rate == 200:
        st2[n].decimate(2)
        st2[n].decimate(10)
        st2[n].decimate(10)
        st2[n].decimate(10)
    elif st2[n].stats.sampling_rate == 125:
        st2[n].decimate(5)
        st2[n].decimate(5)
        st2[n].decimate(5)
        st2[n].decimate(10)
    elif st2[n].stats.sampling_rate == 100:
        st2[n].decimate(10)
        st2[n].decimate(10)
        st2[n].decimate(10)
    elif st2[n].stats.sampling_rate == 80:
        st2[n].decimate(10)
        st2[n].decimate(8)
        st2[n].decimate(10)
    elif st2[n].stats.sampling_rate == 50:
        st2[n].decimate(10)
        st2[n].decimate(5)
        st2[n].decimate(10)
    elif st2[n].stats.sampling_rate == 40:
        st2[n].decimate(10)
        st2[n].decimate(4)
        st2[n].decimate(10)
    elif st2[n].stats.sampling_rate == 20:
        st2[n].decimate(10)
        st2[n].decimate(2)
        st2[n].decimate(10)
    elif st2[n].stats.sampling_rate == 1:
        st2[n].decimate(10)
    try:
        st2[n].remove_response(inventory=inv)
    except:
        continue
st2.detrend()
st2.detrend(type="demean")

In [ ]:
# Now do the plotting

sc = 1.3  # Amplitude scaling factor
#fig = plt.figure(figsize=(5.11, 3.14))
fig = plt.figure(figsize=(11, 12))
ax = plt.subplot()
st3 = st2.copy()

for tr in tqdm(st3):
    sta = tr.stats.station
    try:
        sta_inv = inv.select(station=sta)[0][0]
    except:
        continue
    dist = locations2degrees(ev_lat, ev_lon, sta_inv.latitude, sta_inv.longitude)
    if f"{tr.stats.network}.{tr.stats.station}" in ["II.BFO", "DK.SCO"]:#, "II.ALE", "GB.CWF"]:
        lw = 1.5
        alpha = 0.99
        zorder = 10
    else:
        lw = 0.15
        alpha=0.55
        zorder = 0
    if tr.stats.network == "NL":
        continue
    tr.detrend()
    tr.detrend(type="demean")
    tr.filter("lowpass", freq=12e-3)
    tr.filter("lowpass", freq=12e-3)
    tr.filter("highpass", freq=10e-3)    #tr.taper(max_percentage=0.05)
    tr.taper(max_percentage=0.05)
    tr.trim(ot-15*60, ot+(2*60*60))
    tr.taper(max_percentage=0.05)


    #if sta == "CWF":
    #    print(dist)
    #_, az, baz = gps2dist_azimuth(ev_lat, ev_lon, sta_inv.latitude, sta_inv.longitude)
    try:
        #pass
        #ax.plot(tr.times(reftime=ot)/60, dist + (sc*np.abs(tr.data)/np.max(np.abs(tr.data))), lw=0.1, alpha=0.2, c="k")
        ax.plot(tr.times(reftime=ot)/60, dist + (sc*tr.data/np.max(tr.data)), lw=lw, alpha=alpha, c="k", zorder=zorder) #c=scm6.lipari(dist/(1.2*60)), zorder=zorder)
        #if tr.stats.network == "GB":
        #    print(tr.stats.starttime)

        #ax.plot(tr.times(reftime=ot)/60, dist + (sc*(env(tr.data))/np.max(env(tr.data))), lw=0.1, alpha=0.2, c="k")
    except:
        continue
    #ax.text(0, dist, tr.get_id(), fontsize=6)
    #if f"{tr.stats.network}.{tr.stats.station}" in ["II.BFO", "DK.SCO"]: #, "II.ALE", "GB.CWF"]:
    #    ax.text(89, dist, f"{tr.stats.network}.{tr.stats.station}",path_effects=[pe.withStroke(linewidth=2.5, foreground="white")], ha="right", va="center", zorder=20)


#phase = "S"
#x_all = np.arange(0, 90, 1)
#xx_all = []
#t_all = []
#for x in x_all:
#    arr = model.get_travel_times(source_depth_in_km=0, distance_in_degree=x, phase_list=[phase])
#    if arr:
#        t_all.append(arr[0].time/60)
#        xx_all.append(x)
#    if x == 29:
#        print(arr[0].time)
#ax.plot(t_all, xx_all, label=phase, alpha=0.7, ls="--")

#phase = "SS"
#xx_all = []
#t_all = []
#for x in x_all:
#    arr = model.get_travel_times(source_depth_in_km=0, distance_in_degree=x, phase_list=[phase])
#    if arr:
#        t_all.append(arr[0].time/60)
#        xx_all.append(x)
#ax.plot(t_all, xx_all, label=phase, alpha=0.7, ls="--")

#phase = "4.1kmps"
#xx_all = []
#t_all = []
#for x in x_all:
#    arr = model.get_travel_times(source_depth_in_km=0, distance_in_degree=x, phase_list=[phase])
#    if arr:
#        t_all.append(arr[0].time/60)
#        xx_all.append(x)
#ax.plot(t_all, xx_all, label=phase)

#ax.legend()

#ax.set_ylim([0, 60])
ax.set_xlabel("Minutes since 2023-09-16T12:35")
ax.set_ylabel("Distance (degrees)")


ax.margins(x=0)
ax.margins(y=0)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_ylim([0, 180])
ax.set_xlim([-5, 110])

plt.savefig(f"Record-{T2}-{T1}_alldists.png", bbox_inches="tight", dpi=300)